In [ ]:
from datasets import load_dataset
from tqdm import tqdm
tqdm.pandas()

import openai
openai.api_key = ''

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")
dataset

In [ ]:
td = dataset['train']
td

## 비속어 사전을 활용하여 비속어가 포함된 문장 선별

In [ ]:
fwords = fwords = [
    "개년",
    "ㄱㅅㄲ",
    "개새끼",
    "개씨발",
    "개씹",
    "개족새",
    "개좆",
    "또라이",
    "미친",
    "ㅁㅊ",
    "병신",
    "ㅂㅅ",
    "븅신",
    "븅딱",
    "빠가",
    "뻐큐",
    "새끼",
    "ㅅㄲ",
    "십장생",
    "쌍년",
    "쌍놈",
    "씨발",
    "ㅆㅂ",
    "ㅅㅂ",
    "슈발",
    "시발",
    "스발",
    "싸발",
    "쓰발"
    "씨부랄",
    "씹년",
    "씹새",
    "씹새끼",
    "ㅆ새끼",
    "씹창",
    "존나",
    "좆까",
    "좆망", 
    "좆",
    "좆밥",
    "지랄",
    "ㅈㄹ",
    "ㅈㄴ",
    "졸라",
    "좆같",
    "ㅈ같"
]

In [ ]:
from datasets import load_dataset
import pandas as pd

def find_fwords(example):
    included_fwords = [fword for fword in fwords if fword in example['text']]
    return {'included_fwords': included_fwords}

updated_dataset = dataset['train'].map(find_fwords)

# filter 함수를 사용하여 특정 fwords를 포함하는 문장 필터링
def contains_fword(example):
    return len(example['included_fwords']) > 0

filtered_dataset = updated_dataset.filter(contains_fword)
df= pd.DataFrame(filtered_dataset)
df.head()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base-v2022')

In [ ]:
df_aug = df.copy()
df_aug

In [ ]:
def aug_gpt(example):
    
    encoded = tokenizer.tokenize(example['text'])
    example['words'] = encoded
    prompt = f'''
    레이블은 'O','B-FW','I-FW' 이렇게 세가지가 있어.
    'FW'는 비속어를 의미하고 'B'는 비속어가 시작되는 토큰, 'I'는 비속어의 시작을 제외한 토큰, 'O'태그는 비속어가 아닌 토큰을 의미해.
    주어진 예시처럼 문장 속 비속어를 찾아서 레이블링 해줘.

    ### 예시:
    text: ['씨', '##바', '##알', '.', '.', '노무', '##노무', '술', '##프', '##노', '.', '.', '.', '오늘', '저녁', '##은', '꽂', '##등', '##심이', '##다', '##ㅠ', '##ㅜ']
    label: ('B-FW', 'I-FW', 'I-FW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O')

    text: ['서열', '##1위', '##네', '좆', '##냥', '##이', '찍소리도', '##못함', '##ㅋ']
    label: ('O', 'O', 'O', 'B-FW', 'O', 'O', 'O', 'O', 'O')

    text: ['오', 'ㄱ', '##ㅊ', '##노', '?', '?', '박', '하나', '심어', '##놓은', '##다음', '##에', '존나', '##패', '##는거']
    label: ('O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FW', 'O', 'O')

    text: ['아', '이', '개', '##병', '##신', '##새끼', '##씨', '##발', '##ㅋㅋ']
    label: ('O', 'O', 'O', 'B-FW', 'I-FW', 'I-FW', 'B-FW', 'I-FW', 'O')

    text: ['쓰', '##벌', '새끼들아', '보지', '##마', '보면서', '왜', '작가', '##들에게', '상처를', '주나']
    label: ('B-FW', 'I-FW', 'B-FW', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O')
    
    text:{encoded}
    label:
    '''
    

    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            temperature=0,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        label = response.choices[0].message['content']
        
    
    except Exception as e:
        print(e)
        label = 'error'
    example['label'] = label
    return example


In [ ]:
labeled_df = df_aug.progress_apply(aug_gpt,axis=1)
labeled_df

### 에러가 난 경우 제외

In [ ]:
correct_df = labeled_df[labeled_df['label']!='error']
correct_df

In [ ]:
gpt= correct_df[['text','label','words']]
gpt = gpt.rename(columns= {'text':'content','label':'labels'})
gpt

### 끝까지 출력되지 않은 경우 제외

In [ ]:
gpt = gpt[gpt['labels'].str.endswith(')')]
gpt

### 레이블의 길이와 토큰 길이가 다른경우 제외

In [ ]:
gpt = gpt[gpt['labels'].apply(lambda x:len(eval(x)))==gpt['words'].apply(lambda x:len(eval(x)))]
gpt

In [ ]:
gpt.to_excel('new_aug_gpt.xlsx')